# Deterministic Uncertainty Sets

In [1]:
import cvxpy as cp
import scipy as sc
from sklearn import datasets
import numpy as np
import numpy.random as npr
import numpy.testing as npt
import torch
import pandas as pd
import lropt
import matplotlib.pyplot as plt
RTOL = 1e-04
ATOL = 1e-04
import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive"]

### Formulating the uncertainty set

The first step is to formulate the uncertainty set. There are multiple options, with their associated input parameters. The uncertainty set can be formulated within the uncertain parameter, which takes in the size of the uncertainty and an uncertainty set as parameters.

#### Ellipsoidal uncertainty: $\{u \mid \| Au + b \|_p \leq \rho \}$

- $\rho$ : float, optional  
  * Ellipsoid scaling. Default 1.0. 
- $p$ : integer, optional  
  * Order of the norm. Default 2.
- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.


In [2]:
# Ellipsoidal uncertainty set example
m = 5
data = np.random.normal(0,1,size = (100,m))
ellip_u = lropt.UncertainParameter(m,
                                   uncertainty_set = lropt.Ellipsoidal(p = 2, 
                                                                       rho=2., b = np.mean(data, axis = 0)))

#### Mean Robust uncertainty: $\{ u = (v_1,\dots,v_K)  \mid  \sum_{k=1} ^K w_k \| A_k(v_k - \bar{d}_k) \|_p^{\text{power}} \le \rho \}$ 

- $K$: int
  * Number of clusters. Default 1.
- data : np.array
  * Data to be clustered. The $K$ cluster centers are denoted $\bar{d}_k$ for $k = 1, \dots,K$.
- $\rho$ : float, optional  
  * Ellipsoid scaling. Default 1.0. 
- $p$ : integer, optional  
  * Order of the norm. Default 2.
- power: integer, optional
  * Power of the norm. Default 1. 
- $A_k$ : np.array, optional
  * Scaling matrix for u, for each $k$. Default identity matrix.
- Train: boolean, optional
  * Whether or not $A_k$ should be trained using given data. Default True.


In [3]:
# size of uncertain parameter
m = 5
# Generate data
data = np.random.normal(0,1,size = (100,m))

mro_u = lropt.UncertainParameter(m,
                                 uncertainty_set = lropt.MRO(K = 1, rho=2., data = data, train = False))

#### Box uncertainty: $\{ u \mid\| Au + b\|_\infty \leq \rho \}$ 

- $\rho$ : float, optional  
  * Box scaling. Default 1.0.
- $A$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b$ : np.array, optional
  * Relocation vector for u. Default None.


In [4]:
# Box uncertainty set example
box_u = lropt.UncertainParameter(m, 
                                 uncertainty_set = lropt.Box(rho=2.))

#### Budget uncertainty: $\{u \mid \| A_1u + b_1 \|_\infty \leq \rho_1, \| A_2u + b_2 \|_1 \leq \rho_2\}$ 

- $\rho_1$ : float, optional  
  * Box scaling. Default 1.0. 
- $\rho_2$ : float, optional
  * 1-norm scaling. Default 1.0.
- $A_1$, $A_2$ : np.array, optional
  * Scaling matrix for u. Default identity matrix.
- $b_1$, $b_2$ : np.array, optional
  * Relocation vector for u. Default None.


In [5]:
# Budget uncertainty set example
budget_u = lropt.UncertainParameter(m, 
                                    uncertainty_set = lropt.Budget(rho1=2., 
                                                                   rho2 = 1.))

#### Polyhedral uncertainty: $\{ u \mid Du \leq d\}$

- $D$ : np.array  
- $d$ : np.array  

In [6]:
# Polyhedral uncertainty set example
poly_u = lropt.UncertainParameter(m, 
                                  uncertainty_set = lropt.Polyhedral(D = np.ones((3,m)), d = np.array([1,2,3])))

###  Formulating the Robust Problem

We can now fomulate the Robust Problem, treating the uncertain parameter as a regular parameter.

#### Example 1: Affine transformed LP, ellipsoidal uncertainty and mean robust uncertainty.
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pu+a)^Tx \leq  10,\\
\end{array}
$$
where $c$, $P$, and $a$ are constants, and $u$ is the uncertain parameter from the ellipsoidal set above, where $b$ is nonzero.

In [7]:
#restate the ellipsoidal set
ellip_u = lropt.UncertainParameter(m,
                                  uncertainty_set = lropt.Ellipsoidal(p = 2, 
                                                                      rho=2., b = -np.mean(data, axis = 0)))
n = 4

# formulate cvxpy variable
x_r = cp.Variable(4)

# formulate problem constants
P = 3. * np.eye(m)[:n, :]
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(c@x_r)

# formulate constraints
constraints = [(P@ellip_u +a)@ x_r <= 10]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x_r.value)


LRO objective value:  -1.6519071154902834 
LRO x:  [-1.28865213 -1.0510689  -0.00152114 -0.66158881]


We note that this is equivalent to using the MRO uncertainty set with $K = 1$ and power = 1.

In [8]:
#
mro_u = lropt.UncertainParameter(m,
                                 uncertainty_set = lropt.MRO(rho=2., K = 1, data = data, train = False))
n = 4

# formulate cvxpy variable
x_m = cp.Variable(4)

# formulate objective
objective = cp.Minimize(c@x_m)

# formulate constraints
constraints = [(P@mro_u +a)@ x_m <= 10]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("MRO objective value: ", prob_robust.objective.value, "\nMRO x: ", x_m.value)


MRO objective value:  -1.6519071151229305 
MRO x:  [-1.28865203 -1.05106898 -0.00152115 -0.66158886]


We compare the above with its explicit reformulation and solution
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx - b^T(P^Tx) + \rho\|P^Tx\|_2 \leq  10,\\
\end{array}
$$

In [9]:
# formulate cvxpy variable
x = cp.Variable(4)

# formulate objective
objective = cp.Minimize(c@x)

# formulate constraints
constraints = [a@x + np.mean(data, axis = 0)@(P.T@x) + 2*cp.norm(P.T@x,2) <= 10]

# formulate problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()

print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -1.6519071095609226 
Cvxpy x:  [-1.28861583 -1.05109806 -0.00152409 -0.66160572]


We see that they provide the same solution.

#### Example 2: Multiple uncertain terms, box uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & P(Au+ b)^Tx  \leq  10 + 2(Au + b)^Tx\\
& x \geq 0,\quad x \leq 1,
\end{array}
$$
where again $c$, $P$, and $a$ are constants, and $u$ is the uncertain parameter from a box uncertainty set. We note that the uncertain parameter can appear more than once in the uncertain constraint. 

Note that using the box uncertainty set is equivalent to using the ellipsoidal uncertainty set with $p = \infty$

In [10]:
#formulate using the box set
box_u = lropt.UncertainParameter(m, 
                                 uncertainty_set = lropt.Box(rho=2.))
n = 5

# formulate cvxpy variable
x_box = cp.Variable(n)

# formulate problem constants
P = 3*np.random.rand(n,m)
c = np.random.rand(n)
center = 0.5*np.ones(m)
side = 0.1*np.array([1,2,3,4,5])
A = np.diag(0.5*side)
b = center

# formulate objective
objective = cp.Minimize(-c@x_box)

# formulate constraints
constraints = [(P@A@box_u+ P@b)@ x_box - 2*(A@box_u + b)@x_box <= 10 , x_box >=0, x_box<=1]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
newprob = prob_robust.dualize_constraints()
newprob.solve()
print("Box objective value: ", newprob.objective.value, "\nBox x: ", x_box.value)

#formulate using the ellipsoidal set
ellip_u = lropt.UncertainParameter(m,
                                   uncertainty_set = lropt.Ellipsoidal(p = np.inf, 
                                                                       rho=2., A = np.eye(m)))
n = 5

# formulate cvxpy variable
x_el = cp.Variable(n)

# formulate objective
objective = cp.Minimize(-c@x_el)

# formulate constraints
constraints = [P@(A@ellip_u + b)@ x_el  <= 10+(2*A@ellip_u+ 2*b)@x_el, x_el >=0, x_el<=1]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()

print("Ellipsoidal objective value: ", prob_robust.objective.value, "\nEllipsoidal x: ", x_el.value)

# assert x values are equal
npt.assert_allclose(x_box.value, x_el.value, rtol=RTOL, atol=ATOL)


Box objective value:  -1.4078979589735572 
Box x:  [7.29768334e-11 3.03296876e-01 1.00000000e+00 1.00000000e+00
 3.35504847e-12]
Ellipsoidal objective value:  -1.4078979589735567 
Ellipsoidal x:  [7.29771431e-11 3.03296876e-01 1.00000000e+00 1.00000000e+00
 3.35507240e-12]


We compare the above with its explicit reformulation and solution
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pb)^Tx - 2b^Tx + \rho\|A^TP^Tx - 2A^Tx\|_1 \leq  10,\\
\end{array}
$$

In [11]:
# formulate using cvxpy
x_cvxpy = cp.Variable(5)
# formulate objective
objective = cp.Minimize(-c@x_cvxpy)

# formulate constraints
constraints = [(P@b)@x_cvxpy - 2*b@x_cvxpy + 2*cp.norm((P@A).T@x_cvxpy - 2*A.T@x_cvxpy,p=1)<= 10, x_cvxpy>=0, x_cvxpy<=1]

# formulate problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()

print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_box.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -1.407897958860878 
Cvxpy x:  [4.90973587e-10 3.03296878e-01 9.99999999e-01 1.00000000e+00
 2.55184680e-11]


#### Example 3: Multiple uncertain constraints, polyhedral uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (P_1u + a)^Tx  \leq  10\\
& (P_2u)^Tx  \leq  5\\
& x \leq 5,
\end{array}
$$
where $c$, $P_1$, $P_2$, and $a$ are constants, and $u$ is the uncertain parameter from a polyhedral uncertainty set

In [12]:
#restate the ellipsoidal set 
D =  np.vstack((np.eye(m), -2*np.eye(m)))
d = np.concatenate((0.1*np.ones(m), 0.1*np.ones(m)))
poly_u = lropt.UncertainParameter(m, 
                                  uncertainty_set = lropt.Polyhedral(D = D, d = d))
n = 4
# formulate cvxpy variable
x_r = cp.Variable(n)

# formulate problem constants
P1 = 0.5 * np.eye(m)[:n, :]
P2 = 3*np.random.rand(n,m)
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(-c@x_r)

# formulate constraints
constraints = [(P1@poly_u +a)@ x_r <=10, x_r<=5]
constraints += [(P2@poly_u)@ x_r <= 5]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve

# solve
prob_robust.solve()
print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x_r.value)

LRO objective value:  -6.847182231773523 
LRO x:  [-5.86505292  5.          5.          4.99999999]


We compare it with its direct reformulation, where each uncertain constraint is treated independently,
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx + w_1^Td  \leq  10\\
& w_1^TD = P_1^Tx\\
& w_2^Td  \leq  5\\
& w_2^TD = P_2^Tx\\
& w_1, w_2 \geq 0\\
& x \leq 5,
\end{array}
$$
where $w_1$, $w_2$ are new variables introduced.

In [13]:
# formulate using cvxpy
x_cvxpy = cp.Variable(n)
w1 = cp.Variable(2*m)
w2 = cp.Variable(2*m)
# formulate objective
objective = cp.Minimize(-c@x_cvxpy)

# formulate constraints
constraints = [a@x_cvxpy + w1@d <= 10]
constraints += [w1@D == P1.T@x_cvxpy]
constraints += [w2@d <= 5]
constraints += [w2@D == P2.T@x_cvxpy]
constraints += [w1>=0, w2 >= 0, x_cvxpy <=5]
# formulate Robust Problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)

Cvxpy objective value:  -6.847182239193556 
Cvxpy x:  [-5.86505291  5.          5.          5.        ]


#### Example 4: Direct comparison constraints, budget uncertainty
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pu + a)^Tx  \leq  10\\
& x \geq P_1u,
\end{array}
$$
where $c$, $P$,and $a$ are constants, and $u$ is the uncertain parameter from a budet uncertainty set. 
We can include constraints that relates directly the values of $x$ and $u$, as in the second constraint.

In [19]:
#restate the ellipsoidal set
budget_u = lropt.UncertainParameter(m, 
                                    uncertainty_set = lropt.Budget(rho1=2., 
                                                                   rho2 = 1., A1 = np.eye(m), A2 = np.eye(m)))
n = 4
# formulate cvxpy variable
x_r = cp.Variable(n)

# formulate problem constants
P = 3. * np.eye(m)[:n, :]
P1 = 3*np.random.rand(n,m)
a = 0.1 * np.random.rand(n)
c = np.random.rand(n)

# formulate objective
objective = cp.Minimize(c@x_r)

# formulate constraints
constraints = [(P@budget_u +a)@ x_r <= 10]
constraints += [x_r >= P1@budget_u]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x_r.value)


LRO objective value:  5.261330257320349 
LRO x:  [2.9843025  2.98430251 2.9843025  2.9843025 ]


We compare this with its direct reformulation,
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & a^Tx + \rho_1\|y\|_1 + \rho_2\|P^Tx - y\|_\infty  \leq  10\\
& -x_i + \rho_1\|z_i\|_1 + \rho_2\|(P_1^T)_i - z_i\|_\infty \leq 0, \quad i = 1,\dots,n
\end{array}
$$
where $(P_1^T)_i$ is the $i$'th row of $P_1$.

In [20]:
# formulate cvxpy variables
x_cvxpy = cp.Variable(n)
y = cp.Variable(m)
z = cp.Variable((n,m))

# formulate objective
objective = cp.Minimize(c@x_cvxpy)

# formulate constraints
constraints = [a@x_cvxpy + 2*cp.norm(y,1) + cp.norm(P.T@x_cvxpy-y,np.inf) <= 10]
for i in range(n):
  constraints += [-x_cvxpy[i] + 2*cp.norm(z[i],1) + cp.norm(P1.T@(np.eye(n)[i]) - z[i],np.inf) <= 0]

# formulate Robust Problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  4.413832675527443 
Cvxpy x:  [2.05171805 1.42413308 2.81831985 2.9843025 ]


AssertionError: 
Not equal to tolerance rtol=0.0001, atol=0.0001

Mismatched elements: 3 / 4 (75%)
Max absolute difference: 1.56016943
Max relative difference: 1.09552221
 x: array([2.984303, 2.984303, 2.984303, 2.984303])
 y: array([2.051718, 1.424133, 2.81832 , 2.984303])

#### Example 5: Quadratic uncertainty, ellipsoidal set
We solve the problem
$$
\begin{array}{ll}
\text{minimize} & \tau\\
\text{subject to}  & -(1/2)\sum_{i=1}^n (u^TP_iu)x_i  \leq  \tau\\
& \mathbf{1}^Tx = 3\\
& x \geq 0, \quad x \leq 1
\end{array}
$$
where each $P_i\in \mathbf{R}^{n\times n}$ is a symmetric positive definite matrix, and $u$ is the uncertain parameter from an ellipsoidal uncertainty set. 

In [21]:
# formulate uncertainty set
n = 5
u = lropt.UncertainParameter(n,
                             uncertainty_set=lropt.Ellipsoidal(p=2, rho=0.5))
# formulate cvxpy variables
x_r = cp.Variable(n)
t = cp.Variable()

# formulate problem constants
P = {}
P_inv = {}
for i in range(n):
    P[i] = datasets.make_spd_matrix(n, random_state=i)
    P_inv[i] = sc.linalg.sqrtm(np.linalg.inv(P[i]))

# formulate objective
objective = cp.Minimize(t)

# formulate constraints 
constraints = [cp.sum([-0.5*lropt.quad_form(u, P[i]*x_r[i]) for i in range(n)]) <= t]
constraints += [cp.sum(x_r)==4]
constraints += [x_r >= 0.6, x_r <= 1]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x_r.value)

LRO objective value:  -2.09735042339787e-10 
LRO x:  [0.79877007 0.80135793 0.80079074 0.79877817 0.80030309]


We compare this with the direct reformulation,
\begin{equation}
	\begin{array}{lll}
		&\text{minimize} & \tau \\
		&\text{subject to} & (1/2)\sum_{i=1}^n (z_i^TP_i^{-1}z_i)/x_i + \rho\|z\|_2 \le \tau\\
		&& \sum_{i=1}^n z_i = z,\\
&& \mathbf{1}^Tx = 3\\
&& x \geq 0, \quad x \leq 1
	\end{array}
\end{equation}
where we have $\gamma \in \mathbf{R}^m, z \in \mathbf{R}^n, z_i \in \mathbf{R}^n$, $i = 1,\dots,n$, as auxiliary variables. 

In [22]:
# formulate using cvxpy
x_cvxpy = cp.Variable(n)
t = cp.Variable()
z = cp.Variable(n)
y = cp.Variable((n,n))
gam = cp.Variable(n)

# formulate objective
objective = cp.Minimize(t)

# formulate constraints
constraints = [cp.sum([cp.quad_over_lin(y[i]@P_inv[i],2*x_cvxpy[i]) for i in range(n)]) + 0.5*cp.norm(z,2) <= t]
constraints += [cp.sum(y, axis = 1) == z]
constraints += [cp.sum(x_cvxpy) == 4]
constraints += [x_cvxpy >= 0.6, x_cvxpy <= 1]

# formulate problem 
prob_cvxpy = cp.Problem(objective, constraints)
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -2.119588106628019e-10 
Cvxpy x:  [0.79880041 0.80132141 0.80077192 0.79880835 0.80029791]


#### Example 6: Max of affine uncertainty, ellipsoidal set, mean robust set
We solve the problem
$$
\begin{array}{ll}
		\text{minimize} & \tau \\
		\text{subject to } & \max\{a^Tx - d^Tx, a^Tx - d^T(3u+3)\} \leq \tau,\\
\end{array}
$$
where $a$ and $d$ are constants, and $u$ is the uncertain parameter from an ellipsoidal uncertainty set. 

In [24]:
# formulate uncertainty set
n = 5
u = lropt.UncertainParameter(n,
                             uncertainty_set=lropt.Ellipsoidal(p=2, rho=1, b = -np.mean(data, axis = 0)))
# formulate cvxpy variables
x_r = cp.Variable(n)
t = cp.Variable()

# formulate constants
a = npr.uniform(1,4,n)
d = a + npr.uniform(2,5,n)

# formulate objective
objective = cp.Minimize(t)

# formulate constraints 
constraints = [cp.maximum(a@x_r - d@x_r, a@x_r - d@(3*u+3)) <= t]
constraints += [x_r >= 0]


# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("LRO objective value: ", prob_robust.objective.value, "\nLRO x: ", x_r.value)

LRO objective value:  -24.204947337131998 
LRO x:  [1.66109909e-08 2.04794281e+01 7.49201439e-09 7.39865821e-09
 3.18003418e-08]


We again note that this is equivalent to using the MRO uncertainty set with $K= 1$ and power = 1.

In [25]:
# formulate uncertainty set
n = 5
u = lropt.UncertainParameter(n,
                             uncertainty_set=lropt.MRO(K = 1, data = data,p=2,
                                                       rho=1, train = False))
# formulate cvxpy variables
x_m = cp.Variable(n)
t = cp.Variable()

# formulate objective
objective = cp.Minimize(t)

# formulate constraints 
constraints = [cp.maximum(a@x_m - d@x_m, a@x_m - d@(3*u+3)) <= t]
constraints += [x_m >= 0]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()
print("MRO objective value: ", prob_robust.objective.value, "\nMRO x: ", x_m.value)

MRO objective value:  -24.20494733865909 
MRO x:  [1.40071965e-08 2.04794281e+01 6.00534032e-09 5.88589368e-09
 2.58236577e-08]


In [22]:
# formulate using cvxpy
x_cvxpy = cp.Variable(n)
t = cp.Variable()

# formulate objective
objective = cp.Minimize(t)

# formulate constraints
constraints = [a@x_cvxpy -d@x_cvxpy + cp.norm(3*d,2) <= t]
constraints += [a@x_cvxpy -3*d@np.ones(n) + np.mean(data, axis = 0)@(-3*d) + cp.norm(3*d,2) <= t]
constraints += [x_cvxpy >= 0]

# formulate problem 
prob_cvxpy = cp.Problem(objective, constraints)
prob_cvxpy.solve()
print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_r.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -30.79802922217572 
Cvxpy x:  [1.63082452e-09 6.21752185e-10 7.86787336e-10 1.85258181e+01
 2.87047513e-10]
